In [1]:
import json
import numpy as np
from fmi_mlc import fmi_gym
from config import parameter as PARAMS
from mqtt_publisher import MQTTPublisher
import time
# 1) Inicializar entorno del edificio con store_data=True
env = fmi_gym(PARAMS)

# 2) Crear publisher para Influx (uso de funciones base)
pub = MQTTPublisher(host="mosquitto", port=1883)

# 3) Reset y obtención de la primera observación
obs = env.reset()
last = env.data.iloc[-1].to_dict()
ts = last['time']

# 4) Publicar primera observación base
obs_dict = {name: last[name] for name in PARAMS['observation_names']}
pub.publish_observations_base(obs_dict, ts)


done = False
while not done:
    # 5) Acción por defecto: 25 grados en todas las zonas
    default_action = np.full(len(PARAMS['action_names']), 0.0, dtype=np.float32)
    act_dict = {name: float(default_action[i])
                for i, name in enumerate(PARAMS['action_names'])}

    # 6) Publicar acción base
    pub.publish_actions_base(act_dict, ts)
    

    # 7) Avanzar un paso en el entorno
    obs, reward, done, info = env.step(default_action)
    last = env.data.iloc[-1].to_dict()
    ts = last['time']

    # 8) Publicar siguiente observación base
    obs_dict = {name: last[name] for name in PARAMS['observation_names']}
    pub.publish_observations_base(obs_dict, ts)
    

    # 9) Publicar recompensas individuales base
    reward_dict = {r: last[r] for r in PARAMS['reward_names']}
    pub.publish_rewards_base(reward_dict, ts)

    time.sleep(0.001)
    

# 10) Cierre
print("✅ Base Sim — Episode finished.")
pub.close()

Reading input and weather file for preprocessor program.
The IDF version of the input file ///tmp/JModelica.org/jm_tmpg2en44wn//resources//Office_IDF.idf starts with 9
Successfully finish reading weather file.
This is the Begin Month: 6
This is the Day of the Begin Month: 2
This is the End Month: 6
This is the Day of the End Month: 30
Day of week was left blank in input file.
This is the New Day of Week:  
Running EPMacro...
ExpandObjects Started.
No expanded file generated.
ExpandObjects Finished. Time:     0.015
EnergyPlus Starting
EnergyPlus, Version 9.6.0-4b123cf80f, YMD=2025.06.18 07:50
Initializing Response Factors
Calculating CTFs for "PROJECT INTERNAL FLOOR"
Calculating CTFs for "PROJECT PARTITION"
Calculating CTFs for "PROJECT PARTITION_REV"
Calculating CTFs for "PROJECT INTERNAL DOOR"
Calculating CTFs for "WALL_LAYER"
Calculating CTFs for "ROOF_LAYER"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Encl

[WARNING][Slave] [warning][FMU status:Warning] fmiInitializeSlave: The StopTimeDefined parameter is set to 0. This is not valid. EnergyPlus FMU requires a stop time and will use the stop time 15638400.000000 which is provided.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The sockfd is 52.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The port number is 43385.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: This hostname is d2a70a720a3e.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: TCPServer Server waiting for clients on port: 43385.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The number of input variables is 4.

[INFO][Slave] [ok][FMU status:OK] fmiInitializeSlave: The number of output variables is 10.

[INFO][Slave] [ok][FMU status:OK] Get input file from resource folder ///tmp/JModelica.org/jm_tmpg2en44wn//resources//.

[INFO][Slave] [ok][FMU status:OK] Searching for following pattern .idf

[INFO][Slave] [ok][FMU status:OK] Read director

Updating Shadowing Calculations, Start Date=06/22/2002
Updating Detailed Daylighting Factors, Start Date=06/22
Continuing Simulation at 06/22/2002 for UNTITLED (01-01:31-12)
✅ Base Sim — Episode finished.
